# Proper nouns with or without definite articles (Nestle1904GBI)

**Work in progress**

## Table of content <a class="anchor" id="TOC"></a>
* <a href="#bullet1">1 - Introduction</a>
    * <a href="#bullet1x1">1.1 - Why is this relevant?</a>
    * <a href="#bullet1x2">1.2 - Translating into Text-Fabric queries</a>
* <a href="#bullet2">2 - Load Text-Fabric app and data</a>
* <a href="#bullet3">3 - Performing the queries</a>
    * <a href="#bullet3x1">3.1 - Determine articles in combination with proper nouns</a>
    * <a href="#bullet3x2">3.2 - Show a syntax tree</a>
* <a href="#bullet4">4 - Footnotes and attribution</a>
    * <a href="#bullet4x1">4.1 Textual variants</a>
* <a href="#bullet5">5 - Required libraries</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to TOC](#TOC)

In this Jupyter NoteBook we will examine the use of proper nouns with or without definite articles in the Greek New Testament.

Proper nouns in Biblical Greek are typically used without a definite article when they function as names. For example: Πέτρος εἶπε... (Peter said...)

## 1.1 - Why is this relevant? <a class="anchor" id="bullet1x1"></a>
##### [Back to TOC](#TOC)

In the Greek New Testament, the presence or absence of definite articles with proper nouns can carry specific grammatical and semantic implications. The pressence or absence of the article can make a difference for a few reasons, e.g.:

* Identifiability: The presence of a definite article (e.g., ὁ, ἡ, τὸ) before a proper noun indicates that the noun refers to a specific, identifiable entity. It often implies that the noun is known to both the speaker and the audience. On the other hand, the absence of the definite article suggests a more general or less specific reference.

* Emphasis: The use or omission of the definite article can be employed for emphasis or to highlight certain aspects of the proper noun. When the definite article is present, it draws attention to the individual or highlight particular qualities or characteristics. Without the definite article, the focus may shift more to the general category or class to which the noun belongs.

* Unique versus Common Names: Proper nouns in Greek can be classified as either unique or common names. Unique names refer to specific individuals or places, such as Ἰησοῦς or Ἱεροσόλυμα. These often appear with the definite article when referring to a well-known individual or location. Common names, on the other hand, are more general and do not require the definite article. For example, ἄνθρωπος can refer to any man in general.

* Anaphoric References: In Greek, the presence or absence of the definite article can indicate anaphoric references, referring back to a previous mention. The use of the definite article can signal that the proper noun is referring to someone or something previously introduced or discussed, providing continuity and clarity in the discourse.

It would be of interest if the Text-Fabric dataset could provide clues to how an article is uses.


## 1.2 - Translating into Text-Fabric queries <a class="anchor" id="bullet1x2"></a>
##### [Back to TOC](#TOC)

The following examples gather pressence and absence of definive article with proper nouns.

# 2 - Load Text-Fabric app and data <a class="anchor" id="bullet2"></a>
##### [Back to TOC](#TOC)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Loading the Text-Fabric code
# Note: it is assumed Text-Fabric is installed in your environment
from tf.fabric import Fabric
from tf.app import use

In [3]:
# load the N1904 app and data
N1904 = use ("tonyjurg/Nestle1904GBI", version="0.4", hoist=globals())

**Locating corpus resources ...**

The requested app is not available offline
	~/text-fabric-data/github/tonyjurg/Nestle1904GBI/app not found


The requested data is not available offline
	~/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf/0.4 not found


   |     0.16s T otype                from ~/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf/0.4
   |     1.85s T oslots               from ~/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf/0.4
   |     0.59s T word                 from ~/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf/0.4
   |     0.52s T verse                from ~/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf/0.4
   |     0.51s T after                from ~/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf/0.4
   |     0.52s T chapter              from ~/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf/0.4
   |     0.59s T book                 from ~/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf/0.4
   |      |     0.05s C __levels__           from otype, oslots, otext
   |      |     1.60s C __order__            from otype, oslots, __levels__
   |      |     0.07s C __rank__             from otype, __order__
   |      |     2.20s C __levUp__            from otype, oslots, __rank__
   |      |     1.4

Name,# of nodes,# slots / node,% coverage
book,27,5102.93,100
chapter,260,529.92,100
sentence,5720,24.09,100
verse,7943,17.35,100
clause,16124,8.54,100
phrase,72674,1.90,100
word,137779,1.00,100


In [4]:
# The following will push the Text-Fabric stylesheet to this notebook (to facilitate proper display with notebook viewer)
N1904.dh(N1904.getCss())

# 3 - Performing the queries <a class="anchor" id="bullet3"></a>
##### [Back to TOC](#TOC)

## 3.1 - Determine articles in combination with proper nouns<a class="anchor" id="bullet3x1"></a>
##### [Back to TOC](#TOC)

This query selects each phrase containing (at least) two words: the article (a) and the proper noun (b).

The query without congruence conditions yealds 9500 results. Adding congruence conditions limits the results step by step: adding [`case`](../features/case.md#README) limits it to 8521, adding [`num`](../features/num.md#README) to 8320, and adding [`gn`]((../features/gn.md#README)) further to 8067.

In [5]:
# Define the query template
ArticulatedQuery = '''
phrase 
  a:word sp=det
  b:word sp=noun type=Proper
a .case. b
a .nu. b
a .gn. b
'''

# The following will create a list containing ordered tuples consisting of node numbers of the items as they appear in the query
ArticulatedResult = N1904.search(ArticulatedQuery)

  0.26s 8067 results


The list ArticulatedResult contains ordered tuples with the node numbers of the found phrase, det, and noun combinations. The data can now further be analyzed. The following table provides insight in which phrase types most frequently contain an articulated proper noun. This table still needs to deal with multiple findings in the same phrase.

In [8]:
# Library to format table
from tabulate import tabulate

ResultDict = {} # dictionary to store the found results

# the following can be set/unset to filter out duplicate occurances within the same phrase
FilterOutDuplicates = True
NodeList = list() # list to store node number of each phrase found (to filter out duplicates)

for (phrase,det,noun) in ArticulatedResult:  # phrase contains the node number
    PhraseFunction=F.phrasefunctionlong.v(phrase)
    # Check if this node already exists in NodeList
    if phrase in NodeList:
        print ("it is found")
    else: 
    
    # Check if this PhraseFunction already exists in ResultDict
    if PhraseFunction in ResultDict:
         # If it exists, add the count to the existing value
         ResultDict[PhraseFunction]+=1
    else:
         # If it doesn't exist, initialize the count as the value
         ResultDict[PhraseFunction]=1
    
# Convert the dictionary into a list of key-value pairs and sort it according to frequency
UnsortedTableData = [[key, value] for key, value in ResultDict.items()]
TableData= sorted(UnsortedTableData, key=lambda row: row[1], reverse=True)

# Produce the table
headers = ["Phrase function","Frequency"]
print(tabulate(TableData, headers=headers, tablefmt='fancy_grid'))

╒══════════════════════════╤═════════════╕
│ Phrase function          │   Frequency │
╞══════════════════════════╪═════════════╡
│ Predicate function       │        5770 │
├──────────────────────────┼─────────────┤
│ Subject function         │        1078 │
├──────────────────────────┼─────────────┤
│ Adverbial function       │         638 │
├──────────────────────────┼─────────────┤
│ Object function          │         450 │
├──────────────────────────┼─────────────┤
│ Indirect Object function │         103 │
├──────────────────────────┼─────────────┤
│ Verbal function          │          10 │
├──────────────────────────┼─────────────┤
│ Second Object function   │           8 │
├──────────────────────────┼─────────────┤
│ TBD                      │           8 │
├──────────────────────────┼─────────────┤
│ Verbal Copula function   │           2 │
╘══════════════════════════╧═════════════╛


There is a dificulty to determine how many different phrases are involved..... The verselocation below points to the start of the phrase (clearest indicated by the high number for Luke 3:23 which starts a long geneology).

In [6]:
PhrasesDict = {}
for (phrase,det,noun) in ArticulatedResult:
    location=T.sectionFromNode(phrase)
    # Check if this location already exists in PhrasesDict
    if location in PhrasesDict:
         # If it exists, it is a duplicate: add the count to the existing value
         PhrasesDict[location]+=1
    else:
         # If it doesn't exist, mark it down
         PhrasesDict[location]=1
    
# Convert the dictionary into a list of key-value pairs and sort it according to frequency
UnsortedTableData = [[key, value] for key, value in PhrasesDict.items()]
TableData= sorted(UnsortedTableData, key=lambda row: row[1], reverse=True)

# In this example the table will be truncated 
max_rows = 10  # Set your desired number of rows here
TruncatedTable = TableData[:max_rows]

# Produce the table
headers = ["verselocation","Frequency"]
print(tabulate(TruncatedTable, headers=headers, tablefmt='fancy_grid'))

# Add a warning using markdown (API call A.dm) allowing it to be printed in bold type
N1904.dm("**Warning: table truncated!**")

╒════════════════════╤═════════════╕
│ verselocation      │   Frequency │
╞════════════════════╪═════════════╡
│ ('Luke', 3, 23)    │        5624 │
├────────────────────┼─────────────┤
│ ('Mark', 3, 18)    │          23 │
├────────────────────┼─────────────┤
│ ('Matthew', 10, 4) │          12 │
├────────────────────┼─────────────┤
│ ('Mark', 15, 40)   │          10 │
├────────────────────┼─────────────┤
│ ('Mark', 14, 33)   │           9 │
├────────────────────┼─────────────┤
│ ('Mark', 15, 47)   │           9 │
├────────────────────┼─────────────┤
│ ('Mark', 16, 1)    │           9 │
├────────────────────┼─────────────┤
│ ('Mark', 3, 17)    │           8 │
├────────────────────┼─────────────┤
│ ('Luke', 6, 17)    │           8 │
├────────────────────┼─────────────┤
│ ('Luke', 24, 10)   │           8 │
╘════════════════════╧═════════════╛


**Warning: table truncated!**

## 3.2 - Show a syntax tree<a class="anchor" id="bullet3x2"></a>
##### [Back to TOC](#TOC)

The following displays the syntax tree for the first result.

In [7]:
N1904.show(ArticulatedResult, start=1, end=1, condensed=True, multiFeatures=True)

# 4 - Footnotes and attribution <a class="anchor" id="bullet4"></a>
##### [Back to TOC](#TOC)

Thanks to Prof. Willem van Peursen (VU) for pointing me to this question and raising the following interesting issue in respect to the use of a definite article: 
> In Muraoka’s *Why Read the Bible in the Original Languages*,[^1] I found the following examples:
(...)
The definite article e.g. “the truth will set you free” (John 8:32; TJ) how to translate in Japanese, which doesn’t have the definite article.

[^1] Muraoka, Takamitsu. Why Read the Bible in the Original Languages? (Leuven: Peeters Publishers, 2020), 71.

## 4.1 - Textual variants <a class="anchor" id="bullet4x1"></a>
##### [Back to TOC](#TOC)

The research in articulated proper nouns is also relevant in relation to variant readings. Consider for exemple Acts 18:1.

NA28:
> Μετὰ ταῦτα χωρισθεὶς ἐκ τῶν Ἀθηνῶν ἦλθεν εἰς Κόρινθον

Stephanus Textus Receptus 1550: 
> Μετὰ **δὲ** ταῦτα χωρισθεὶς **ὁ Παῦλος** ἐκ τῶν Ἀθηνῶν ἦλθεν εἰς Κόρινθον

# 5 - Required libraries <a class="anchor" id="bullet5"></a>
##### [Back to TOC](#TOC)

The scripts in this notebook require (beside `text-fabric`) the following Python libraries to be installed in the environment:

    tabulate

You can install any missing library from within Jupyter Notebook using either`pip` or `pip3`.